In [268]:
# !pip install catboost

In [269]:
# !pip install xgboost

In [270]:
# !pip install lightgbm

In [271]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [272]:
## Load libraries
import pandas as pd
import numpy as np

In [273]:
# Display options
pd.set_option('display.max_columns',50)

## Read data

In [274]:
train_data = pd.read_csv('updated_train.csv')
test_data = pd.read_csv('updated_test.csv')

In [275]:
train_data.shape
train_data.head(2)

(948, 43)

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,winner,winner_id,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,bat_strength_team1,bat_strength_team2,series_type,bat_strike_rate_team1,bat_strike_rate_team2,bat_strike_rate_team,bowl_strength_team1,bowl_strength_team2,economy_team1,economy_team2,team1_strength,team2_strength,team_strength,economy_team,wins_ground_team1,wins_ground_team2,wins_ground_team,ground_toss_vs_wins_team1,ground_toss_vs_wins_team2,ground_toss_vs_wins_team_norm_diff
0,9331181,Ba,11283,9373356.0:7857520.0:4232164.0:4566540.0:329940...,Hl Ph,12634,3500958.0:4231751.0:2735081.0:2035102.0:369833...,Hl Ph,12634,Hl Ph,field,Hr Ct Sm Ie,Indore,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.0,100.0,157.178571,38.047457,36.622575,other_domestic,113.787802,114.165394,0.988038,32.551047,27.145611,6.738024,6.860950,35.299252,31.884093,1.073445,0.857151,0.0,1.0,0.0,0.999001,0.999001,0.000
1,8797060,Ed,20,2089079.0:6139370.0:2076192.0:62432.0:2083409....,Wt Is,41,4690258.0:4069666.0:4230127.0:1942317.0:161392...,Ed,20,Wt Is,field,Kn Ol Bn Bs,Bridgetown,2022-01-23,day/night match,Ed tr of Wt Is,2021/22,1406,1.285714,1.952381,156.0,50.0,103.500000,46.755378,46.884421,other_domestic,126.022329,129.297584,0.967189,35.593313,42.438386,7.891324,7.586442,41.174345,44.661404,0.901732,0.919045,0.0,0.0,0.0,0.000000,0.999001,-0.999


In [276]:
test_data.shape
test_data.head(2)

(271, 41)

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,bat_strength_team1,bat_strength_team2,series_type,bat_strike_rate_team1,bat_strike_rate_team2,bat_strike_rate_team,bowl_strength_team1,bowl_strength_team2,economy_team1,economy_team2,team1_strength,team2_strength,team_strength,economy_team,wins_ground_team1,wins_ground_team2,wins_ground_team,ground_toss_vs_wins_team1,ground_toss_vs_wins_team2,ground_toss_vs_wins_team_norm_diff
0,9250275,Jy,28594,7438955.0:8271969.0:8369661.0:3685247.0:259025...,Ud Ss of Aa,90,2436944.0:8117500.0:6703528.0:3010748.0:161387...,Ud Ss of Aa,field,Bo Ac Cb,Bulawayo,2022-07-11,day match,Ud Ss of Aa tr of Ze,2022,3226,0.000000,0.019608,NaN,0.00,NaN,1.400000,38.428630,other_domestic,0.000000,121.905254,0.000000,0.000000,15.281200,0.000000,8.353618,0.700000,26.854915,0.02513,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,9262189,Ga An Ws,36084,3715690.0:6818622.0:4069666.0:393014.0:4690188...,Ja Ts,36098,4690258.0:3761246.0:8464385.0:5742470.0:315072...,Ja Ts,field,Pe Sm Ga,Providence,2022-09-21,night match,Cn Pr Le,2022,13915,0.615385,0.344262,151.285714,66.67,153.5,46.172005,47.917872,other_domestic,122.717675,119.555481,1.017935,34.391343,35.032277,7.103168,7.920621,40.281674,41.475075,0.94836,0.796264,0.0,0.0,0.0,0.0,0.0,0.0


#### Create dependent column

In [277]:
train_data['winner_01'] = train_data.apply(lambda x: 1 if (x['team2']==x['winner']) else 0, axis=1)

#### Pre process

In [278]:
## Toss winner to numerical - 1 if team2 wins, else 0

train_data['toss_winner_01'] = np.where(train_data['toss winner']==train_data['team2'], 1, 0)
test_data['toss_winner_01'] = np.where(test_data['toss winner']==test_data['team2'], 1, 0)

In [279]:
## Toss decision - categorical - 1 if winner bats, 0 otherwise

train_data['toss_decision_01'] = np.where(train_data['toss decision']=='bat', 1, 0)
test_data['toss_decision_01'] = np.where(test_data['toss decision']=='bat', 1, 0)

#### Selecting relevant columns

In [280]:
X,y = train_data[['toss_winner_01','toss_decision_01','team_count_50runs_last15','team_winp_last5','team1only_avg_runs_last15','team1_winp_team2_last15','ground_avg_runs_last15','team_strength', 'economy_team', 'wins_ground_team', 'ground_toss_vs_wins_team_norm_diff']], train_data['winner_01']

In [281]:
X_test = test_data[X.columns.tolist()]

#### Imputation

In [282]:
X_nans = X.isna().sum().reset_index()
X_nans[X_nans[0]!=0]

X_test_nans = X_test.isna().sum().reset_index()
X_test_nans[X_test_nans[0]!=0]

,index,0
4,team1only_avg_runs_last15,21
6,ground_avg_runs_last15,53


,index,0
4,team1only_avg_runs_last15,7
6,ground_avg_runs_last15,10


In [283]:
X.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

C:\Users\Anish Mulay\AppData\Local\Temp\ipykernel_30692\1698769224.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(0, inplace=True)
C:\Users\Anish Mulay\AppData\Local\Temp\ipykernel_30692\1698769224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(0, inplace=True)


In [284]:
X_nans = X.isna().sum().reset_index()
X_nans[X_nans[0]!=0]

X_test_nans = X_test.isna().sum().reset_index()
X_test_nans[X_test_nans[0]!=0]

,index,0


,index,0


### Stats

In [285]:
X['toss_winner_01'].value_counts()
X['toss_decision_01'].value_counts()
X['team_count_50runs_last15'].describe()

1    620
0    328
Name: toss_winner_01, dtype: int64

0    617
1    331
Name: toss_decision_01, dtype: int64

count    948.000000
mean       1.202602
std        1.008793
min        0.083333
25%        0.686678
50%        1.000000
75%        1.400000
max       11.000000
Name: team_count_50runs_last15, dtype: float64

## Model

#### GBM

In [286]:
from sklearn.ensemble import GradientBoostingClassifier

In [287]:
# # user-defined parameters

# algo_name = 'GradientBoostingClassifier'
# is_ensemble = 'no'
# n_trees = 10
# depth = 2
# lr = 0.1

##### Train

In [288]:
# clf_gbm = GradientBoostingClassifier(n_estimators = n_trees, max_depth = depth, learning_rate = lr).fit(X,y)

In [289]:
# train_data['y_pred_01'] = clf_gbm.predict(X)
# test_data['y_pred_01'] = clf_gbm.predict(X_test)

### HYPERPARAMTER TUNING

In [290]:
import pandas as pd
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)



In [314]:
import catboost as cb
from sklearn.metrics import mean_squared_error
import optuna
from sklearn.metrics import accuracy_score
from optuna.samplers import TPESampler

# def objective(trial):
#     params = {
#         "iterations": 1000,
#         "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
#         "depth": trial.suggest_int("depth", 1, 10),
#         "subsample": trial.suggest_float("subsample", 0.05, 1.0),
#         "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
#         "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
#     }

#     model = cb.CatBoostClassifier(**params, silent=True)
#     model.fit(X_train, y_train)
#     predictions = model.predict(X_val)
#     rmse = mean_squared_error(y_val, predictions, squared=False)
#     return rmse

# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=30)


def objective(trial):
    model = cb.CatBoostClassifier(
        iterations=trial.suggest_int("iterations", 100, 1000),
        learning_rate=trial.suggest_float("learning_rate", 0.095, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 10),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        od_wait=trial.suggest_int("od_wait", 10, 50),
        verbose=False
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    return accuracy_score(y_val, y_pred)

sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="catboost", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=50)


[I 2024-06-19 00:19:16,515] A new study created in memory with name: catboost
[I 2024-06-19 00:19:17,353] Trial 0 finished with value: 0.5157894736842106 and parameters: {'iterations': 475, 'learning_rate': 0.09857569280069847, 'depth': 4, 'l2_leaf_reg': 1.0551779964424746e-05, 'bootstrap_type': 'Bayesian', 'random_strength': 2.0931628460945333e-07, 'bagging_temperature': 0.923385947687978, 'od_type': 'Iter', 'od_wait': 26}. Best is trial 0 with value: 0.5157894736842106.
[I 2024-06-19 00:19:19,563] Trial 1 finished with value: 0.5263157894736842 and parameters: {'iterations': 585, 'learning_rate': 0.0970647963632925, 'depth': 8, 'l2_leaf_reg': 1.10795595820296e-06, 'bootstrap_type': 'Bayesian', 'random_strength': 0.7999391045172093, 'bagging_temperature': 0.27387593197926163, 'od_type': 'IncToDec', 'od_wait': 32}. Best is trial 1 with value: 0.5263157894736842.
[I 2024-06-19 00:19:21,131] Trial 2 finished with value: 0.5473684210526316 and parameters: {'iterations': 226, 'learning_rat

In [292]:
trial = study.best_trial

In [293]:
# algo_name = 'CatboostClassifier'
# is_ensemble = 'no'
# n_trees = 10
# depth = 2
# lr = 0.1

In [294]:
## cat boost

from catboost import CatBoostClassifier
# clf = CatBoostClassifier(
#     iterations = 100,
#     learning_rate = learning_rate,
#     depth= depth,
#     loss_function='MultiClass',
#     verbose= True,  # Set to True if you want to see training progress
#     # subsample=subsample,
#     colsample_bylevel=colsample_bylevel,
#     min_data_in_leaf=min_data_in_leaf
# )

clf = CatBoostClassifier(
    **trial.params,
    loss_function='MultiClass',
    verbose=False
)

clf.fit(X_train,y_train)


In [295]:
## xg boost
# from xgboost import XGBClassifier
# from sklearn.multiclass import OneVsRestClassifier  # For multi-class classification

# # Initialize the XGBoost classifier
# clf = OneVsRestClassifier(XGBClassifier(
#     n_estimators=100,  # similar to iterations in CatBoost
#     learning_rate=0.1,
#     max_depth=6,
#     objective='multi:softprob',  # for multi-class classification
#     verbosity=0  # Set to 1 or 2 if you want to see training progress
# ))

# # Fit the model
# clf.fit(X, y)

# # Predict on test data
# test_data['y_pred_01'] = clf.predict(X_test)

In [296]:
## light gbm
# from lightgbm import LGBMClassifier
# from sklearn.multiclass import OneVsRestClassifier

# # Initialize the LightGBM classifier
# clf = OneVsRestClassifier(LGBMClassifier(
#     n_estimators=100,  # similar to iterations in CatBoost
#     learning_rate=0.01,
#     max_depth=6,
#     objective='multiclass',  # for multi-class classification
#     num_class=len(set(y)),  # specify the number of classes
#     verbose=-1  # Set to 0 or higher if you want to see training progress
# ))

# # Fit the model
# # Note: LightGBM prefers its own DMatrix format, but it can also work with numpy arrays or pandas DataFrames
# clf.fit(X, y)

# # Predict on test data
# test_data['y_pred_01'] = clf.predict(X_test)

# # If you want probability predictions:
# # test_data['y_pred_proba'] = clf.predict_proba(X_test)

In [297]:
train_data['y_pred_01'] = clf.predict(X)
test_data['y_pred_01'] = clf.predict(X_test)

In [298]:
from sklearn.metrics import classification_report

In [299]:
# Train accuracy
# print(classification_report(y, clf_gbm.predict(X), labels=[0,1]))
# print(classification_report(y, clf.predict(X), labels=[0,1]))
print(classification_report(y_val, clf.predict(X_val), labels=[0,1]))

              precision    recall  f1-score   support

           0       0.57      0.48      0.52        50
           1       0.51      0.60      0.55        45

    accuracy                           0.54        95
   macro avg       0.54      0.54      0.54        95
weighted avg       0.54      0.54      0.54        95



In [300]:
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val, clf.predict(X_val), labels=[0,1])
print(f'ROC AUC: {roc_auc}')

ROC AUC: 0.54


In [301]:
print(accuracy_score(y_val, clf.predict(X_val)))

0.5368421052631579


In [302]:
# train_data['win_pred_score'] = clf_gbm.predict_proba(X)[:,1]
# test_data['win_pred_score'] = clf_gbm.predict_proba(X_test)[:,1]
train_probas = clf.predict_proba(X)
test_probas = clf.predict_proba(X_test)

train_data['win_pred_score'] = [probs[pred] for probs, pred in zip(train_probas, train_data['y_pred_01'])]
test_data['win_pred_score'] = [probs[pred] for probs, pred in zip(test_probas, test_data['y_pred_01'])]

##### Getting win_pred_score corresponding to winner team

In [303]:
train_data['win_pred_score'] = np.where( (train_data['y_pred_01']==0), (1-train_data['win_pred_score']), train_data['win_pred_score'])
test_data['win_pred_score'] = np.where( (test_data['y_pred_01']==0), (1-test_data['win_pred_score']), test_data['win_pred_score'])

##### Getting winner_team_id from custom dependent column

In [304]:
train_data['win_pred_team_id'] = np.where( (train_data['y_pred_01']==0), (train_data['team1_id']), train_data['team2_id'])
test_data['win_pred_team_id'] = np.where( (test_data['y_pred_01']==0), (test_data['team1_id']), test_data['team2_id'])

#### Feature importance

In [305]:
# df_feat_importance = pd.DataFrame({'feat_name':X.columns.tolist(), 'model_feat_imp_train':clf_gbm.feature_importances_}).sort_values(by='model_feat_imp_train', ascending=False)\
#                                                                                                                         .reset_index(drop=True).head(10)
# df_feat_importance

df_feat_importance = pd.DataFrame({'feat_name':X.columns.tolist(), 'model_feat_imp_train':clf.feature_importances_}).sort_values(by='model_feat_imp_train', ascending=False)\
                                                                                                                        .reset_index(drop=True).head(10)
df_feat_importance

,feat_name,model_feat_imp_train
0,team_count_50runs_last15,15.133404
1,ground_avg_runs_last15,13.037632
2,economy_team,12.007073
3,team1only_avg_runs_last15,11.968724
4,team_strength,11.728719
5,team_winp_last5,11.091961
6,ground_toss_vs_wins_team_norm_diff,8.771069
7,wins_ground_team,6.955277
8,team1_winp_team2_last15,6.271261
9,toss_decision_01,1.892806


## File preparation

#### File 1

In [306]:
train_data['dataset_type'] = 'train'
test_data['dataset_type'] = 'r1'

In [307]:
## refactor

df_file1 = pd.concat([test_data[['match id','dataset_type','win_pred_team_id','win_pred_score',] + list(df_feat_importance['feat_name'].head(10))], \
                     train_data[['match id','dataset_type','win_pred_team_id','win_pred_score',] + list(df_feat_importance['feat_name'].head(10))]])

renaming_dict = {}
for i,col in enumerate(list(df_feat_importance['feat_name'].head(10))):
    renaming_dict[col] = f'indep_feat_id{i+1}'
df_file1.rename(columns=renaming_dict, inplace=True)

for i in range(1,11):
    if f'indep_feat_id{i}' not in df_file1.columns:
        df_file1[f'indep_feat_id{i}'] = np.nan

df_file1['train_algorithm'] = algo_name
df_file1['is_ensemble'] = is_ensemble
df_file1['train_hps_trees'] = n_trees
df_file1['train_hps_depth'] = depth
df_file1['train_hps_lr'] = lr

In [308]:
df_file1.shape
df_file1.head()

(1219, 19)

,match id,dataset_type,win_pred_team_id,win_pred_score,indep_feat_id1,indep_feat_id2,indep_feat_id3,indep_feat_id4,indep_feat_id5,indep_feat_id6,indep_feat_id7,indep_feat_id8,indep_feat_id9,indep_feat_id10,train_algorithm,is_ensemble,train_hps_trees,train_hps_depth,train_hps_lr
0,9250275,r1,90,0.763108,0.000000,NaN,0.000000,NaN,0.025130,0.019608,0.00000,0.000000,0.00,0,CatboostClassifier,no,10,2,0.1
1,9262189,r1,36098,0.592554,0.615385,153.500,0.796264,151.285714,0.948360,0.344262,0.00000,0.000000,66.67,0,CatboostClassifier,no,10,2,0.1
2,9128776,r1,48334,0.600362,0.842105,179.625,0.871776,171.066667,1.212345,0.753086,-0.00025,0.666667,100.00,1,CatboostClassifier,no,10,2,0.1
3,9586919,r1,36112,0.569691,0.285714,155.500,0.788943,166.000000,0.982988,1.487805,-0.00025,0.266667,75.00,0,CatboostClassifier,no,10,2,0.1
4,9128538,r1,30414,0.437546,2.375000,164.125,0.877328,169.933333,1.235356,0.310330,-0.99900,0.000000,0.00,0,CatboostClassifier,no,10,2,0.1


#### File 2

In [309]:
feature_desc = {'team_count_50runs_last15':'Ratio of number of 50s by players in team1 to number of 50s by players in team2 in last 15 games',\
               'team_winp_last5':'Ratio of team1\'s win % to team2\'s win % in last 5 games',\
               'ground_avg_runs_last15':'average runs scored in the ground in last 15 games',\
               'team1_winp_team2_last15':'Team1\'s win percentage against Team2 in last 15 games',\
               'team1only_avg_runs_last15':'team1\'s avg inning runs in last 15 games',\
                
               'season_num' : 'Numerical form of season. Takes 1 for oldest season and increases for latest seasons.',\
               'toss_winner_01' : 'Toss winner to numerical - 1 if team2 wins, else 0',\
               'toss_decision_01' : 'Toss decision - categorical - 1 if winner bats, 0 otherwise',\
                'bat_strength_team1' : 'calculate the team\'s batting strength, 0.2 weight is assigned to batting strength calculated using batting average and strike rate, 0.4 for a run scored, 0.4 for partnership ', 
                'bat_strength_team2' : 'calculate the team\'s batting strength, 0.2 weight is assigned to batting strength calculated using batting average and strike rate, 0.4 for a run scored, 0.4 for partnership ',
                # 'bowl_strength_team1': 'bowling strength of the team1',
                # 'bowl_strength_team2': 'bowling strength of the team2',
                'team_strength' :'strength of the team1',
                # 'team2_strength' :'strength of the team2',
                'economy_team': 'economy strength of the team1',
                # 'economy_team2': 'economy strength of the team2',
                'wins_ground_team' : 'win ratio of the teams at the ground',
                'ground_toss_vs_wins_team_norm_diff' : 'bat first vs bowl first wins team1/team2',
                }

In [310]:
# df_feat_importance.rename(index={0:'feat_id'}, inplace=True)
df_file2 = df_feat_importance
df_file2['feat_id'] = [i+1 for i in df_file2.index]
df_file2['feat_rank_train'] = [i+1 for i in df_file2.index]
df_file2 = df_file2.set_index('feat_id')
df_file2['feat_description'] = df_file2['feat_name'].map(feature_desc)

In [311]:
df_file2

,feat_name,model_feat_imp_train,feat_rank_train,feat_description
feat_id,,,,
1,team_count_50runs_last15,15.133404,1,Ratio of number of 50s by players in team1 to ...
2,ground_avg_runs_last15,13.037632,2,average runs scored in the ground in last 15 g...
3,economy_team,12.007073,3,economy strength of the team1
4,team1only_avg_runs_last15,11.968724,4,team1's avg inning runs in last 15 games
5,team_strength,11.728719,5,strength of the team1
6,team_winp_last5,11.091961,6,Ratio of team1's win % to team2's win % in las...
7,ground_toss_vs_wins_team_norm_diff,8.771069,7,bat first vs bowl first wins team1/team2
8,wins_ground_team,6.955277,8,win ratio of the teams at the ground
9,team1_winp_team2_last15,6.271261,9,Team1's win percentage against Team2 in last 1...


## Save

In [312]:
df_file1.to_csv('file1_catboost.csv', index=False)
df_file2.to_csv('file2_catboost.csv')

In [313]:
train_data = pd.read_csv('data/663e2b6d54457_train_data_with_samplefeatures.csv')
true_winners_id = train_data['winner_id']
pred_winner_id = df_file1['win_pred_team_id']
print(len(train_data))
print(len(true_winners_id))
num_wins = 0

print(pred_winner_id)
print("\n\n\n")
for i in range(len(train_data)):
    print(true_winners_id[i])
    print("pred : ", pred_winner_id[i])
    if true_winners_id[i] == pred_winner_id[i+272]:
        num_wins +=1

accuracy = num_wins/len(train_data)

948
948
0         90
1      36098
2      48334
3      36112
4      30414
       ...  
943    30414
944     9701
945    23869
946    36014
947     7608
Name: win_pred_team_id, Length: 1219, dtype: int64




12634
pred :  0       90
0    11283
Name: win_pred_team_id, dtype: int64
20
pred :  1    36098
1       20
Name: win_pred_team_id, dtype: int64
10576
pred :  2    48334
2     8987
Name: win_pred_team_id, dtype: int64
36084
pred :  3    36112
3    36084
Name: win_pred_team_id, dtype: int64
48341
pred :  4    30414
4    48341
Name: win_pred_team_id, dtype: int64
40424
pred :  5    30435
5    42573
Name: win_pred_team_id, dtype: int64
55
pred :  6    30414
6       55
Name: win_pred_team_id, dtype: int64
47501
pred :  7     9701
7    47501
Name: win_pred_team_id, dtype: int64
12634
pred :  8     7727
8    14860
Name: win_pred_team_id, dtype: int64
55
pred :  9    30428
9       55
Name: win_pred_team_id, dtype: int64
30414
pred :  10    36098
10    30414
Name: win_pred_team_id, dtype: int6

KeyError: 948

In [ ]:
def getAccuracy(df):
    return round(df[df['winner_id'] == df['win_pred_team_id']].shape[0]*100/df.shape[0], 4)

In [ ]:
getAccuracy(df_file1)

KeyError: 'winner_id'